In [1]:
from google.colab import drive
drive.mount("/drive")

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import KFold
import time
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
results_filename = "/drive/MyDrive/ACA/Projeto/Results_GridSearch_TVT/Results_LR2.csv"
results_filename1 = "/drive/MyDrive/ACA/Projeto/Results_GridSearch_TVT/Results_LR2.pkl"
def get_results():
    if os.path.exists(results_filename):
        print("Ficheiro existe")
        results = pd.read_pickle(results_filename1)
    else:
        print("Ficheiro nao existe")
        results = pd.DataFrame( columns=["Type", "Dataset", "Train Size", "Validation Size", "Test Size", "Train F1", "Train Recall", "Train Precision", "Train Accuracy", "Validation F1", "Validation Recall", "Validation Precision", "Validation Accuracy", "Test F1", "Test Recall", "Test Precision", "Test Accuracy", "Params"])
    return results
results = get_results()
results

Ficheiro existe


,Type,Dataset,Train Size,Validation Size,Test Size,Train F1,Train Recall,Train Precision,Train Accuracy,Validation F1,Validation Recall,Validation Precision,Validation Accuracy,Test F1,Test Recall,Test Precision,Test Accuracy,Params
0,LR,Outliers_NoCorr_NoST_PCA.csv,"(72901, 2)","(15465, 2)","(15595, 2)",0.683976,0.519828,0.999838,0.519828,0.681890,0.517426,0.999616,0.517426,0.681154,0.516576,0.999872,0.516576,"{'penalty': 'l2', 'solver': 'newton-cg'}"
1,LR,Outliers_NoCorr_NoST_NoPCA.csv,"(72901, 40)","(15465, 40)","(15595, 40)",0.937453,0.934390,0.942385,0.934390,0.938085,0.934627,0.943733,0.934627,0.933855,0.931196,0.938270,0.931196,"{'penalty': 'l1', 'solver': 'liblinear'}"
2,LR,NoOutliers_NoCorr_NoST_PCA.csv,"(72901, 5)","(15465, 5)","(15595, 5)",0.837902,0.818123,0.865013,0.818123,0.836871,0.816036,0.864652,0.816036,0.834459,0.814171,0.862462,0.814171,"{'penalty': 'l2', 'solver': 'newton-cg'}"
3,LR,NoOutliers_NoCorr_NoST_NoPCA.csv,"(72901, 30)","(15465, 30)","(15595, 30)",0.957373,0.956105,0.959647,0.956105,0.957643,0.956288,0.960243,0.956288,0.955854,0.954601,0.958128,0.954601,"{'penalty': None, 'solver': 'newton-cg'}"
4,LR,NoOutliers_NoCorr_ST_PCA.csv,"(72901, 5)","(15465, 5)","(15595, 5)",0.863910,0.846504,0.885814,0.846504,0.862568,0.844035,0.885921,0.844035,0.857499,0.839372,0.881068,0.839372,"{'penalty': None, 'solver': 'lbfgs'}"


In [3]:
datasets_folder = "/drive/MyDrive/ACA/Projeto/dataset/Clean/Train/"

files = os.listdir(datasets_folder)
print(files)
for filename in files:
    if (
        (results["Type"] == "LR") &
        (results["Dataset"] == filename)
    ).any() == False:
        print(filename)
        df = pd.read_csv(datasets_folder + filename)

        if "Target" in list(df.columns):
            colunas_train = list(df.columns)
            colunas_train.remove("Target")
            
            X = list( df[colunas_train].values )
            y = df["Target"].values
        
        elif "attack_type" in list(df.columns):
            colunas_train = list(df.columns)
            colunas_train.remove("attack_type")

            X = list( df[colunas_train].values )
            y = df["attack_type"].values
        
        else: 
            print(df.columns)

        #==============================================================================================================
        #==== Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.15, shuffle= True)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.175)

        print("\t",np.shape(X_train), "-", np.shape(X_val), np.shape(X_test) )
        #==============================================================================================================
        #==== Train
        
        # parameters
        parameters = {"penalty": ["l2", None, "l1", "elasticnet"],
                      "solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]}

        search = GridSearchCV(LogisticRegression(), parameters, cv=3)
        search.fit(X_train, y_train) # train_model
        print("\t", search.best_params_)
        print("\t", search.best_score_)  

        pred_val = search.predict(X_train)

        f1_score_train = f1_score(pred_val, y_train, average='weighted')
        accuracy_score_train = accuracy_score(pred_val, y_train)
        recall_score_train = recall_score(pred_val, y_train, average='weighted')
        precision_score_train = precision_score(pred_val, y_train, average='weighted')  

        print("\tTrain")
        print("\tF1: ", f1_score_train)
        print("\tAccuracy: ", accuracy_score_train)
        print("\tRecall: ", recall_score_train)
        print("\tPrecision: ", precision_score_train)

        #==============================================================================================================
        #==== Validation  

        pred_val = search.predict(X_val)

        f1_score_val = f1_score(pred_val, y_val, average='weighted')
        accuracy_score_val = accuracy_score(pred_val, y_val)
        recall_score_val = recall_score(pred_val, y_val, average='weighted')
        precision_score_val = precision_score(pred_val, y_val, average='weighted')  

        print("\n\tValidation")
        print("\tF1: ", f1_score_val)
        print("\tAccuracy: ", accuracy_score_val)
        print("\tRecall: ", recall_score_val)
        print("\tPrecision: ", precision_score_val)
        #==============================================================================================================
        #==== Test

        pred_val = search.predict(X_test)

        f1_score_test = f1_score(pred_val, y_test, average='weighted')
        accuracy_score_test = accuracy_score(pred_val, y_test)
        recall_score_test = recall_score(pred_val, y_test, average='weighted')
        precision_score_test = precision_score(pred_val, y_test, average='weighted')  

        print("\n\tTest")
        print("\tF1: ", f1_score_test)
        print("\tAccuracy: ", accuracy_score_test)
        print("\tRecall: ", recall_score_test)
        print("\tPrecision: ", precision_score_test)

        #==============================================================================================================
        #=== Save Results
        results = get_results()
        results.loc[len(results)] = ["LR", filename, np.shape(X_train), np.shape(X_val), np.shape(X_test), 
                                    f1_score_train, recall_score_train, precision_score_train, accuracy_score_train,
                                    f1_score_val, recall_score_val, precision_score_val, accuracy_score_val,
                                    f1_score_test, recall_score_test, precision_score_test, accuracy_score_test,
                                    search.best_params_]
        
        results.to_csv(results_filename)
        results.to_pickle(results_filename1)
    
    else:
        pass



    




['Outliers_NoCorr_NoST_PCA.csv', 'Outliers_NoCorr_NoST_NoPCA.csv', 'NoOutliers_NoCorr_NoST_PCA.csv', 'NoOutliers_NoCorr_NoST_NoPCA.csv', 'NoOutliers_NoCorr_ST_PCA.csv', 'NoOutliers_NoCorr_ST_NoPCA.csv', 'Outliers_NoCorr_ST_PCA.csv', 'Outliers_NoCorr_ST_NoPCA.csv']
NoOutliers_NoCorr_ST_NoPCA.csv
	 (72901, 30) - (15465, 30) (15595, 30)
	 {'penalty': None, 'solver': 'newton-cg'}
	 0.9566535481585007
	Train
	F1:  0.9586364423705452
	Accuracy:  0.9574354261258419
	Recall:  0.9574354261258419
	Precision:  0.9607830441696411

	Validation
	F1:  0.958051387051496
	Accuracy:  0.9568056902683478
	Recall:  0.9568056902683478
	Precision:  0.9604539667755053

	Test
	F1:  0.9574738727390403
	Accuracy:  0.9562680346264828
	Recall:  0.9562680346264828
	Precision:  0.9596668535427452
Ficheiro existe
Outliers_NoCorr_ST_PCA.csv
	 (72901, 5) - (15465, 5) (15595, 5)
	 {'penalty': 'l1', 'solver': 'saga'}
	 0.8409624198205982
	Train
	F1:  0.8679235207249866
	Accuracy:  0.8402491049505494
	Recall:  0.840249104